In [492]:
import pandas as pd
import numpy as np
import xgboost as xgb
import statistics
from statistics import mean
from sklearn.model_selection import GroupKFold
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

In [493]:
#import csv2 

G = pd.read_csv("/Users/sky/Desktop/ca2-dataset.csv")

In [494]:
G

Row       lesson       namea      prod               cell  right  bug  \
0      1353  CTA1Z12ZRAH  stuZ3ilopc  Skill135               R1C1      1    0   
1      1392  CTA1Z12ZRAH  stuZ3ilopc   Skill84    Series1AddCurve      1    0   
2      1393  CTA1Z12ZRAH  stuZ3ilopc  Skill124       Series2Label      1    0   
3      1426  CTA1Z12ZRAH  stuZ3ilopc   Skill24  N265013xstrategic      1    0   
4      1427  CTA1Z12ZRAH  stuZ3ilopc  Skill115               R4C1      0    0   
...     ...          ...         ...       ...                ...    ...  ...   
1740  37890  CTA1Z12ZRAH  stuZzwgcq9  Skill100               R4C3      1    0   
1741  37891  CTA1Z12ZRAH  stuZzwgcq9  Skill108               R5C3      1    0   
1742  37959  CTA1Z12ZRAH  stuZzwgcq9  Skill124       Series1Label      1    0   
1743  37981  CTA1Z12ZRAH  stuZzwgcq9  Skill149               R3C3      1    0   
1744  37984  CTA1Z12ZRAH  stuZzwgcq9  Skill115               R4C1      0    0   

      help  choice  string  ...  Prev3Count-up  Prev5Count-up  recent8help  \
0        0       0       0  ...              0              0            0   
1        0       0       0  ...              1              1            0   
2        0       0       0  ...              0              0            0   
3        0       0       0  ...              0              0            0   
4        0       0       0  ...              0              0            0   
...    ...     ...     ...  ...            ...            ...          ...   
1740     0       0       0  ...              0              0            0   
1741     0       0       0  ...              0              0            0   
1742     0       0       0  ...              0              0            0   
1743     0       0       0  ...              0              0            0   
1744     0       0       0  ...              1              1            0   

       recent5wrong  manywrong-up  asymptoteA-up  asymptoteB-up  Behaviour  \
0                 0             0              0              0    ON TASK   
1                 2             0              0              0    ON TASK   
2                 2             0              0              0    ON TASK   
3                 1             0              0              0    ON TASK   
4                 1             0              0              0    ON TASK   
...             ...           ...            ...            ...        ...   
1740              0             0              0              0    ON TASK   
1741              0             0              0              0    ON TASK   
1742              0             0              0              0    ON TASK   
1743              0             0              0              0   OFF TASK   
1744              1             0              0              0    ON TASK   

        Coder                                          Unique-id  
0     awagner  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
1     awagner  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
2     awagner  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
3     awagner  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
4     awagner  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
...       ...                                                ...  
1740  awagner  awagner-closeloop-ins_iellyr-03.30.2011_at_08:...  
1741  awagner  awagner-closeloop-ins_iellyr-03.30.2011_at_08:...  
1742  awagner  awagner-closeloop-ins_iellyr-03.31.2011_at_08:...  
1743  awagner  awagner-closeloop-ins_iellyr-03.31.2011_at_08:...  
1744  awagner  awagner-closeloop-ins_iellyr-03.31.2011_at_08:...  

[1745 rows x 34 columns]

In [495]:
# demonsrate the detailed information of the dataset2

G.describe()

Row        right          bug    help  choice  string  number  \
count   1745.000000  1745.000000  1745.000000  1745.0  1745.0  1745.0  1745.0   
mean   19558.809742     0.719198     0.046418     0.0     0.0     0.0     0.0   
std    10637.676767     0.449520     0.210450     0.0     0.0     0.0     0.0   
min     1353.000000     0.000000     0.000000     0.0     0.0     0.0     0.0   
25%    10128.000000     0.000000     0.000000     0.0     0.0     0.0     0.0   
50%    19372.000000     1.000000     0.000000     0.0     0.0     0.0     0.0   
75%    28444.000000     1.000000     0.000000     0.0     0.0     0.0     0.0   
max    37984.000000     1.000000     1.000000     0.0     0.0     0.0     0.0   

        point      pknow-1      Pknow-2  ...  helppct-up  wrongpct-up  \
count  1745.0  1745.000000  1745.000000  ...      1745.0  1745.000000   
mean      0.0     0.353169     0.869323  ...         0.0     1.543840   
std       0.0     0.891970     0.255355  ...         0.0     5.571414   
min       0.0    -1.000000     0.195152  ...         0.0     0.000000   
25%       0.0    -1.000000     0.888287  ...         0.0     0.000000   
50%       0.0     0.988464     0.998916  ...         0.0     0.000000   
75%       0.0     0.999999     1.000000  ...         0.0     0.000000   
max       0.0     1.000000     1.000000  ...         0.0    72.000000   

       timeperact-up  Prev3Count-up  Prev5Count-up  recent8help  \
count    1745.000000    1745.000000    1745.000000       1745.0   
mean       12.060059       0.496275       0.657880          0.0   
std        11.286503       0.957295       1.339835          0.0   
min        -1.000000       0.000000       0.000000          0.0   
25%         5.666667       0.000000       0.000000          0.0   
50%         9.111111       0.000000       0.000000          0.0   
75%        14.823529       1.000000       1.000000          0.0   
max       119.000000       3.000000       5.000000          0.0   

        recent5wrong  manywrong-up  asymptoteA-up  asymptoteB-up  
count    1745.000000   1745.000000         1745.0         1745.0  
mean        0.912894      0.061891            0.0            0.0  
std         1.108961      0.241027            0.0            0.0  
min         0.000000      0.000000            0.0            0.0  
25%         0.000000      0.000000            0.0            0.0  
50%         1.000000      0.000000            0.0            0.0  
75%         2.000000      0.000000            0.0            0.0  
max         5.000000      1.000000            0.0            0.0  

[8 rows x 27 columns]

In [496]:
# drop the data "help", "choice","string","number","point","helppct-up","recent8help","asymptoteA-up","asymptoteB-up"
# since the value of all these variables are 0

G1 = G.drop(columns=["help", "choice","string","number","point","helppct-up","recent8help","asymptoteA-up","asymptoteB-up"], axis=1)

In [497]:
G1

Row       lesson       namea      prod               cell  right  bug  \
0      1353  CTA1Z12ZRAH  stuZ3ilopc  Skill135               R1C1      1    0   
1      1392  CTA1Z12ZRAH  stuZ3ilopc   Skill84    Series1AddCurve      1    0   
2      1393  CTA1Z12ZRAH  stuZ3ilopc  Skill124       Series2Label      1    0   
3      1426  CTA1Z12ZRAH  stuZ3ilopc   Skill24  N265013xstrategic      1    0   
4      1427  CTA1Z12ZRAH  stuZ3ilopc  Skill115               R4C1      0    0   
...     ...          ...         ...       ...                ...    ...  ...   
1740  37890  CTA1Z12ZRAH  stuZzwgcq9  Skill100               R4C3      1    0   
1741  37891  CTA1Z12ZRAH  stuZzwgcq9  Skill108               R5C3      1    0   
1742  37959  CTA1Z12ZRAH  stuZzwgcq9  Skill124       Series1Label      1    0   
1743  37981  CTA1Z12ZRAH  stuZzwgcq9  Skill149               R3C3      1    0   
1744  37984  CTA1Z12ZRAH  stuZzwgcq9  Skill115               R4C1      0    0   

       pknow-1   Pknow-2  pchange  ...  howmanywrong-up  wrongpct-up  \
0     0.888287  0.888287        0  ...                0            0   
1    -1.000000  0.206074        1  ...                1            0   
2     0.888287  0.888287        0  ...                0            0   
3     0.999899  0.999899        0  ...                0            0   
4     0.909316  0.909316        0  ...                1            1   
...        ...       ...      ...  ...              ...          ...   
1740  0.988464  0.988464        0  ...                3            0   
1741  0.999712  0.999712        0  ...                1            0   
1742  0.999899  0.999899        0  ...                5            0   
1743  0.999992  0.999992        0  ...                2            0   
1744 -1.000000  0.999997        1  ...                2            2   

      timeperact-up  Prev3Count-up  Prev5Count-up   recent5wrong  \
0         26.000000              0              0              0   
1          7.000000              1              1              2   
2         11.500000              0              0              2   
3          6.000000              0              0              1   
4         14.500000              0              0              1   
...             ...            ...            ...            ...   
1740       0.000000              0              0              0   
1741       9.666667              0              0              0   
1742      19.000000              0              0              0   
1743      39.181818              0              0              0   
1744      41.500000              1              1              1   

      manywrong-up  Behaviour    Coder  \
0                0    ON TASK  awagner   
1                0    ON TASK  awagner   
2                0    ON TASK  awagner   
3                0    ON TASK  awagner   
4                0    ON TASK  awagner   
...            ...        ...      ...   
1740             0    ON TASK  awagner   
1741             0    ON TASK  awagner   
1742             0    ON TASK  awagner   
1743             0   OFF TASK  awagner   
1744             0    ON TASK  awagner   

                                              Unique-id  
0     awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
1     awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
2     awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
3     awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
4     awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
...                                                 ...  
1740  awagner-closeloop-ins_iellyr-03.30.2011_at_08:...  
1741  awagner-closeloop-ins_iellyr-03.30.2011_at_08:...  
1742  awagner-closeloop-ins_iellyr-03.31.2011_at_08:...  
1743  awagner-closeloop-ins_iellyr-03.31.2011_at_08:...  
1744  awagner-closeloop-ins_iellyr-03.31.2011_at_08:...  

[1745 rows x 25 columns]

In [498]:
G2 = G1.describe()

In [499]:
G2

Row        right          bug      pknow-1      Pknow-2  \
count   1745.000000  1745.000000  1745.000000  1745.000000  1745.000000   
mean   19558.809742     0.719198     0.046418     0.353169     0.869323   
std    10637.676767     0.449520     0.210450     0.891970     0.255355   
min     1353.000000     0.000000     0.000000    -1.000000     0.195152   
25%    10128.000000     0.000000     0.000000    -1.000000     0.888287   
50%    19372.000000     1.000000     0.000000     0.988464     0.998916   
75%    28444.000000     1.000000     0.000000     0.999999     1.000000   
max    37984.000000     1.000000     1.000000     1.000000     1.000000   

           pchange         time  timeSDnormed  timelast3SDnormed  \
count  1745.000000  1745.000000   1745.000000        1745.000000   
mean      0.295129    11.097421      0.022172           0.175052   
std       0.456231    16.431074      1.745257           3.119500   
min       0.000000    -1.000000     -1.000000          -2.558877   
25%       0.000000     3.000000     -0.612943          -1.310219   
50%       0.000000     7.000000     -0.275081          -0.569318   
75%       1.000000    13.000000      0.122449           0.418168   
max       1.000000   277.000000     51.631811          37.960244   

       timelast5SDnormed     notright  howmanywrong-up  wrongpct-up  \
count        1745.000000  1745.000000      1745.000000  1745.000000   
mean            0.155511     0.280802         2.865330     1.543840   
std             3.952367     0.449520         5.467657     5.571414   
min            -3.928701     0.000000         0.000000     0.000000   
25%            -1.861942     0.000000         0.000000     0.000000   
50%            -0.882886     0.000000         1.000000     0.000000   
75%             0.680272     1.000000         3.000000     0.000000   
max            40.300047     1.000000        42.000000    72.000000   

       timeperact-up  Prev3Count-up  Prev5Count-up   recent5wrong  \
count    1745.000000    1745.000000    1745.000000    1745.000000   
mean       12.060059       0.496275       0.657880       0.912894   
std        11.286503       0.957295       1.339835       1.108961   
min        -1.000000       0.000000       0.000000       0.000000   
25%         5.666667       0.000000       0.000000       0.000000   
50%         9.111111       0.000000       0.000000       1.000000   
75%        14.823529       1.000000       1.000000       2.000000   
max       119.000000       3.000000       5.000000       5.000000   

       manywrong-up  
count   1745.000000  
mean       0.061891  
std        0.241027  
min        0.000000  
25%        0.000000  
50%        0.000000  
75%        0.000000  
max        1.000000

In [500]:
# Create 1st Feature

G1['time_binary']= 0
G1.loc[G1['time']<= G2.iloc[5]['time'], ['time_binary']] = 1

In [501]:
# Create 2nd Feature

G1['howmanywrong-up_binary']= 0
G1.loc[G1['howmanywrong-up']<= G2.iloc[5]['howmanywrong-up'], ['howmanywrong-up_binary']] = 1

In [502]:
# Create 3rd Feature

G1['timeperact-up_binary']= 0
G1.loc[G1['timeperact-up']<= G2.iloc[5]['howmanywrong-up'], ['timeperact-up_binary']] = 1

In [503]:
# Create 4th Feature

G1['Pknow-2_binary']= 0
G1.loc[G1['Pknow-2']<= G2.iloc[5]['Pknow-2'], ['Pknow-2_binary']] = 1

In [504]:
# Create 5th Feature

G1['Prev5Count-up_binary']= 0
G1.loc[G1['Prev5Count-up']<= G2.iloc[5]['Prev5Count-up'], ['Prev5Count-up_binary']] = 1

In [505]:
# Create 6th Feature

G1[['time_z-score']] = (G1[['time']]- G2.iloc[1]['time'])/G2.iloc[2]['time']

In [506]:
# Create 7th Feature

G1[['howmanywrong-up_z-score']] = (G1[['howmanywrong-up']]- G2.iloc[1]['howmanywrong-up'])/G2.iloc[2]['howmanywrong-up']

In [507]:
# Create 8th Feature

G1[['timeperact-up_z-score']] = (G1[['timeperact-up']]- G2.iloc[1]['timeperact-up'])/G2.iloc[2]['timeperact-up']

In [508]:
# Create 9th Feature

G1[['Pknow-2_z-score']] = (G1[['Pknow-2']]- G2.iloc[1]['Pknow-2'])/G2.iloc[2]['Pknow-2']

In [509]:
# Create 10th Feature

G1[['Prev5Count-up_z-score']] = (G1[['Prev5Count-up']]- G2.iloc[1]['Prev5Count-up'])/G2.iloc[2]['Prev5Count-up']

In [510]:
G3 = G1[['time_binary','howmanywrong-up_binary','timeperact-up_binary','Pknow-2_binary','Prev5Count-up_binary','time_z-score','howmanywrong-up_z-score','timeperact-up_z-score','Pknow-2_z-score','Prev5Count-up_z-score','Unique-id']]

In [511]:
G3

time_binary  howmanywrong-up_binary  timeperact-up_binary  \
0               0                       1                     0   
1               1                       1                     0   
2               0                       1                     0   
3               1                       1                     0   
4               0                       1                     0   
...           ...                     ...                   ...   
1740            1                       0                     1   
1741            0                       1                     0   
1742            0                       0                     0   
1743            0                       0                     0   
1744            0                       0                     0   

      Pknow-2_binary  Prev5Count-up_binary  time_z-score  \
0                  1                     1      0.906975   
1                  1                     0     -0.249370   
2                  1                     1     -0.127650   
3                  0                     1     -0.371091   
4                  1                     1      0.237512   
...              ...                   ...           ...   
1740               1                     1     -0.675392   
1741               0                     1      0.176652   
1742               0                     1      0.846115   
1743               0                     1      1.089556   
1744               0                     0     11.800968   

      howmanywrong-up_z-score  timeperact-up_z-score  Pknow-2_z-score  \
0                   -0.524051               1.235098         0.074263   
1                   -0.341157              -0.448328        -2.597364   
2                   -0.524051              -0.049622         0.074263   
3                   -0.524051              -0.536930         0.511351   
4                   -0.341157               0.216182         0.156616   
...                       ...                    ...              ...   
1740                 0.024630              -1.068538         0.466570   
1741                -0.341157              -0.212058         0.510619   
1742                 0.390418               0.614889         0.511351   
1743                -0.158263               2.403026         0.511714   
1744                -0.158263               2.608420         0.511735   

      Prev5Count-up_z-score                                          Unique-id  
0                 -0.491016  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
1                  0.255345  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
2                 -0.491016  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
3                 -0.491016  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
4                 -0.491016  awagner-closeloop-ins_iellyr-03.30.2011_at_12:...  
...                     ...                                                ...  
1740              -0.491016  awagner-closeloop-ins_iellyr-03.30.2011_at_08:...  
1741              -0.491016  awagner-closeloop-ins_iellyr-03.30.2011_at_08:...  
1742              -0.491016  awagner-closeloop-ins_iellyr-03.31.2011_at_08:...  
1743              -0.491016  awagner-closeloop-ins_iellyr-03.31.2011_at_08:...  
1744               0.255345  awagner-closeloop-ins_iellyr-03.31.2011_at_08:...  

[1745 rows x 11 columns]

In [512]:
# import csv1

X = pd.read_csv("/Users/sky/Desktop/ca1-dataset.csv")

In [513]:
# DataSets in CA#2

X4 = X.drop(columns=["Avghelp", "Avgchoice","Avgstring","Avgnumber","Avgpoint","Avghelppct-up","Avgrecent8help","AvgasymptoteA-up","AvgasymptoteB-up","Avgnotright"], axis=1)

In [514]:
# Merge DataSet1 and Dataset2

L = pd.merge(X4,G3,on='Unique-id')

In [515]:
# replace the "Y" and "N" to number 1 and 0 in column "OffTask"

K = X["OffTask"].replace(["Y", "N"], [1, 0])

K_1 = L["OffTask"].replace(["Y", "N"], [1, 0])

In [516]:
# Drop "Unique-id","namea","OffTask" and we can get final Dataset2

L3 = L.drop(columns=["Unique-id","namea","OffTask"], axis=1)

In [517]:
L3

Avgright  Avgbug  Avgpchange    Avgtime  AvgtimeSDnormed  \
0     1.000000     0.0    0.000000  12.000000         0.446098   
1     1.000000     0.0    0.000000  12.000000         0.446098   
2     1.000000     0.0    0.000000   7.500000        -0.260956   
3     1.000000     0.0    0.000000   7.500000        -0.260956   
4     1.000000     0.0    0.000000  25.333333         0.301574   
...        ...     ...         ...        ...              ...   
1740  1.000000     0.0    0.000000  14.000000         0.339286   
1741  1.000000     0.0    0.000000   5.000000        -0.329097   
1742  0.666667     0.0    0.333333  11.000000         0.813424   
1743  0.666667     0.0    0.333333  11.000000         0.813424   
1744  0.666667     0.0    0.333333  11.000000         0.813424   

      Avgtimelast3SDnormed  Avgtimelast5SDnormed  Avghowmanywrong-up  \
0                 0.288201              0.454262                 0.0   
1                 0.288201              0.454262                 0.0   
2                -0.774506              0.000000                 0.0   
3                -0.774506              0.000000                 0.0   
4                -1.529301             -0.508076                 0.0   
...                    ...                   ...                 ...   
1740              1.769959              1.728446                 0.0   
1741              3.106351              4.530658                10.0   
1742              0.602398             -0.155730                 1.0   
1743              0.602398             -0.155730                 1.0   
1744              0.602398             -0.155730                 1.0   

      Avgwrongpct-up  Avgtimeperact-up  ...  time_binary  \
0           0.000000         12.000000  ...            0   
1           0.000000         12.000000  ...            0   
2           0.000000          8.750000  ...            0   
3           0.000000          8.750000  ...            1   
4           0.000000         26.083333  ...            1   
...              ...               ...  ...          ...   
1740        0.000000         17.076923  ...            0   
1741        0.000000         16.777778  ...            1   
1742        0.333333          8.148148  ...            0   
1743        0.333333          8.148148  ...            1   
1744        0.333333          8.148148  ...            0   

      howmanywrong-up_binary  timeperact-up_binary  Pknow-2_binary  \
0                          1                     0               1   
1                          1                     0               1   
2                          1                     0               1   
3                          1                     0               1   
4                          1                     0               1   
...                      ...                   ...             ...   
1740                       1                     0               0   
1741                       0                     0               0   
1742                       1                     0               1   
1743                       1                     0               1   
1744                       1                     0               0   

      Prev5Count-up_binary  time_z-score  howmanywrong-up_z-score  \
0                        1      0.054931                -0.524051   
1                        1      0.054931                -0.524051   
2                        1     -0.188510                -0.524051   
3                        1     -0.249370                -0.524051   
4                        1     -0.553672                -0.524051   
...                    ...           ...                      ...   
1740                     1      0.176652                -0.524051   
1741                     1     -0.371091                 1.304886   
1742                     1      0.054931                -0.341157   
1743                     0     -0.431951                -0.341157   
1744                     1 

In [518]:
# creat a dictionary in CA#2

group_dict = {}
groups = np.array([])

for index, row in L.iterrows():
    student_id = row['namea']
    if student_id not in group_dict:
        group_dict[student_id] = index
    groups = np.append(groups, group_dict[student_id])

In [519]:
# create the batching variable 
# creat 10 folds

gkf = GroupKFold(n_splits=10)

In [520]:
# accuracy, kappa and AUC value based on the Decision Tree in CA#2

clf = tree.DecisionTreeClassifier(min_samples_split=10,random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(clf, L3, K_1, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

kappa_values = list()

for train_index, test_index in gkf.split(L3, K_1, groups=groups):
    X_train = L3.iloc[train_index]
    X_test  = L3.iloc[test_index]
    y_train = K_1[train_index]
    y_test  = K_1[test_index]
    clf = tree.DecisionTreeClassifier(min_samples_split=10,random_state=5)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)
     
print(mean(kappa_values))
print(auc)

Accuracy: 97.88%(0.77%)
0.13123760682525237
0.7412790697674418


In [521]:
# accuracy, kappa and AUC value based on the Naive Bayes in CA#2

gnb = GaussianNB()
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(gnb, L3, K_1, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

kappa_values = list()

for train_index, test_index in gkf.split(L3, K_1, groups=groups):
    X_train = L3.iloc[train_index]
    X_test  = L3.iloc[test_index]
    y_train = K_1[train_index]
    y_test  = K_1[test_index]
    gnb = GaussianNB()
    gnb.fit(X_train, y_train.ravel())
    predictions = gnb.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)
    
print(mean(kappa_values))
print(auc)

Accuracy: 81.43%(22.12%)
0.10935652442509429
0.9883720930232558


In [522]:
# accuracy, kappa and AUC value based on the XGBoosting in CA#2

xgb = XGBClassifier(learning_rate=0.5, n_estimators=200, random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(xgb, L3, K_1, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))


kappa_values = list()

for train_index, test_index in gkf.split(L3, K_1, groups=groups):
    X_train = L3.iloc[train_index]
    X_test  = L3.iloc[test_index]
    y_train = K_1[train_index]
    y_test  = K_1[test_index]
    xgb = XGBClassifier(learning_rate=0.5, n_estimators=200, random_state=5)
    xgb.fit(X_train, y_train)
    predictions = xgb.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)

print(mean(kappa_values))
print(auc)

Accuracy: 98.05%(0.93%)
0.24130830244642007
0.49709302325581395


In [523]:
# accuracy, kappa and AUC value based on the Logistic Regression in CA#2

LR = LogisticRegression(solver='liblinear',random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(LR, L3, K_1, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

kappa_values = list()

for train_index, test_index in gkf.split(L3, K_1, groups=groups):
    X_train = L3.iloc[train_index]
    X_test  = L3.iloc[test_index]
    y_train = K_1[train_index]
    y_test  = K_1[test_index]
    LR = LogisticRegression(solver='liblinear',random_state=5)
    LR.fit(X_train, y_train)
    predictions = LR.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)
     
print(mean(kappa_values))
print(auc)

Accuracy: 97.82%(0.81%)
0.08461734696072938
0.75


In [524]:
# accuracy, kappa and AUC value based on the Random Forest in CA#2

RF = RandomForestClassifier(random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(RF, L3, K_1, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

kappa_values = list()

for train_index, test_index in gkf.split(L3, K_1, groups=groups):
    X_train = L3.iloc[train_index]
    X_test  = L3.iloc[test_index]
    y_train = K_1[train_index]
    y_test  = K_1[test_index]
    RF = RandomForestClassifier(random_state=5)
    RF.fit(X_train, y_train)
    predictions = RF.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)
     
print(mean(kappa_values))
print(auc)

Accuracy: 98.11%(0.89%)
0.06385385181959911
0.5


In [525]:
# DataSets in CA#1

# remove id identifier data "Unique-id" as well as "namea", and the label "off task"
# drop the data "Avghelp", "Avgchoice","Avgstring","Avgnumber","Avgpoint","Avghelppct-up","Avgrecent8help","AvgasymptoteA-up","AvgasymptoteB-up",since the value of all these variables are 0
# drop the column "Avgnotright", since the correlation between the "Avgright" and "Avgnotright" is -1


X3 = X.drop(columns=["Unique-id","namea", "OffTask","Avghelp", "Avgchoice","Avgstring","Avgnumber","Avgpoint","Avghelppct-up","Avgrecent8help","AvgasymptoteA-up","AvgasymptoteB-up","Avgnotright"], axis=1)

In [526]:
# creat a dictionary in CA#1

group_dict = {}
groups = np.array([])

for index, row in X.iterrows():
    student_id = row['namea']
    if student_id not in group_dict:
        group_dict[student_id] = index
    groups = np.append(groups, group_dict[student_id])

In [527]:
# create the batching variable 
# creat 10 folds

gkf = GroupKFold(n_splits=10)

In [528]:
# accuracy, kappa and AUC value based on the Decision Tree in CA#1

clf = tree.DecisionTreeClassifier(min_samples_split=10,random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(clf, X3, K, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

kappa_values = list()

for train_index, test_index in gkf.split(X3, K, groups=groups):
    X_train = X3.iloc[train_index]
    X_test  = X3.iloc[test_index]
    y_train = K[train_index]
    y_test  = K[test_index]
    clf = tree.DecisionTreeClassifier(min_samples_split=10,random_state=5)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)
     
print(mean(kappa_values))
print(auc)

Accuracy: 95.54%(1.88%)
0.2125075951487025
0.5547619047619048


In [529]:
# accuracy, kappa and AUC value based on the Naive Bayes in CA#1

gnb = GaussianNB()
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(gnb, X3, K, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

kappa_values = list()

for train_index, test_index in gkf.split(X3, K, groups=groups):
    X_train = X3.iloc[train_index]
    X_test  = X3.iloc[test_index]
    y_train = K[train_index]
    y_test  = K[test_index]
    gnb = GaussianNB()
    gnb.fit(X_train, y_train.ravel())
    predictions = gnb.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)
    
print(mean(kappa_values))
print(auc)

Accuracy: 78.98%(23.54%)
0.1070665396895294
0.5809523809523811


In [530]:
# accuracy, kappa and AUC value based on the XGBoosting in CA#1

xgb = XGBClassifier(learning_rate=0.5, n_estimators=200, random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(xgb, X3, K, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))


kappa_values = list()

for train_index, test_index in gkf.split(X3, K, groups=groups):
    X_train = X3.iloc[train_index]
    X_test  = X3.iloc[test_index]
    y_train = K[train_index]
    y_test  = K[test_index]
    xgb = XGBClassifier(learning_rate=0.5, n_estimators=200, random_state=5)
    xgb.fit(X_train, y_train)
    predictions = xgb.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)

print(mean(kappa_values))
print(auc)

Accuracy: 96.46%(1.57%)
0.3295338645992353
0.5833333333333334


In [531]:
# accuracy, kappa and AUC value based on the Logistic Regression in CA#1

LR = LogisticRegression(solver='liblinear',random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(LR, X3, K, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

kappa_values = list()

for train_index, test_index in gkf.split(X3, K, groups=groups):
    X_train = X3.iloc[train_index]
    X_test  = X3.iloc[test_index]
    y_train = K[train_index]
    y_test  = K[test_index]
    LR = LogisticRegression(solver='liblinear',random_state=5)
    LR.fit(X_train, y_train)
    predictions = LR.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)
     
print(mean(kappa_values))
print(auc)

Accuracy: 95.80%(1.53%)
0.15046033693230024
0.5761904761904763


In [532]:
# accuracy, kappa and AUC value based on the Random Forest in CA#1

RF = RandomForestClassifier(random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(RF, X3, K, cv=cv_f)

print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

kappa_values = list()

for train_index, test_index in gkf.split(X3, K, groups=groups):
    X_train = X3.iloc[train_index]
    X_test  = X3.iloc[test_index]
    y_train = K[train_index]
    y_test  = K[test_index]
    RF = RandomForestClassifier(random_state=5)
    RF.fit(X_train, y_train)
    predictions = RF.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    auc = roc_auc_score(y_test, predictions)
     
print(mean(kappa_values))
print(auc)

Accuracy: 95.93%(1.81%)
0.2591056343598105
0.5761904761904763
